In [ ]:
from __future__ import division
import pymongo as pm
import numpy as np
import json
import re
import cPickle
import copy
import time
import os
import glob
import pandas as pd

from pylab import *
from numpy import *

conn = pm.MongoClient(port=20809)
DBNAME = conn['during_morph_drawing_recognition']
COLNAME = DBNAME['fmri3.files']
coll=COLNAME
DATADIR = 'neurosketch_data_3'

mdtd = cPickle.load(open('morph_drawing_training_design.pkl'))

# patient ID and worker ID mappings
## exceptions: '1115161_neurosketch', '1116161_neurosketch', '1117161_neurosketch', '1207161_neurosketch'
patient_ids_1 = ['1121161_neurosketch', '1130161_neurosketch', 
'1201161_neurosketch', '1202161_neurosketch', '1203161_neurosketch',
'1206161_neurosketch', '1206162_neurosketch','1206163_neurosketch',
'1207161_neurosketch','1207162_neurosketch']
patient_ids_2 = ['1207162_neurosketch']
patient_ids_3 = ['0110171_neurosketch', '0110172_neurosketch',
'0111171_neurosketch','0112171_neurosketch', '0112172_neurosketch','0112173_neurosketch',
'0113171_neurosketch','0115172_neurosketch','0115174_neurosketch','0117171_neurosketch',
'0118171_neurosketch','0118172_neurosketch','0119171_neurosketch','0119172_neurosketch',
'0119173_neurosketch', '0119174_neurosketch','0120171_neurosketch','0120172_neurosketch',
'0120173_neurosketch','0123171_neurosketch','0123172_neurosketch','0123173_neurosketch',
'0124171_neurosketch','0125171_neurosketch','0125172_neurosketch']

patient_ids = patient_ids_1 + patient_ids_2 + patient_ids_3

all_wIDs = patient_ids
wID_pIDs = zip(patient_ids,all_wIDs)

def get_pID_from_wID(wID):
	pID = [i[0] for i in wID_pIDs if i[1] == wID]
	return pID[0]

def get_phase_index(x):
    if x=='pre':
        return 1
    elif x=='training':
        return 2
    elif x=='post':
        return 3

def get_category_index(x):
    if x=='car':
        return 1
    elif x=='furniture':
        return 2

def get_object_index(morphline,morphnum):
    furniture_axes = ['bedChair', 'bedTable', 'benchBed', 'chairBench', 'chairTable', 'tableBench']
    car_axes = ['limoToSUV','limoToSedan','limoToSmart','smartToSedan','suvToSedan','suvToSmart']  
    furniture_items = ['bed','bench','chair','table']
    car_items = ['limo','sedan','smartcar','SUV']               
    endpoints = getEndpoints(morphline)
    morphnum = float(morphnum)
    whichEndpoint = int(np.round(morphnum/100))
    thing = endpoints[whichEndpoint]
    if morphline in furniture_axes:
        return furniture_items.index(thing)+1
    elif morphline in car_axes:
        return car_items.index(thing)+1

def get_category_name(x):
    if x==1:
        return 'car'
    elif x==2:
        return 'furniture'

def get_object_name(cat_ind,obj_ind):
	furniture_items = ['bed','bench','chair','table']
	car_items = ['limo','sedan','smartcar','SUV']  	
	if cat_ind == 1: # car
		return car_items[obj_ind-1]
	elif cat_ind == 2: # furniture
		return furniture_items[obj_ind-1]

def get_response_index(x):
    furniture_items = ['bed','bench','chair','table']
    car_items = ['limo','sedan','smartcar','SUV']    
    if x in furniture_items:
        return furniture_items.index(x)
    elif x in car_items:
        return car_items.index(x)

def get_sequence_test_worker_list():
	wIDs = COLNAME.find({'wID':{'$ne': ''}}).distinct('wID')
	good_sessions = []
	funky_sessions = []
	good_sessions.append('1020161_neurosketch')
	return good_sessions, funky_sessions

def get_recog_pilot_worker_list():
	wIDs = COLNAME.find({'wID':{'$ne': ''}}).distinct('wID')
	good_sessions = []
	funky_sessions = []
	good_sessions = patient_ids
	return good_sessions, funky_sessions

def get_worker_list():
	wIDs = COLNAME.find({'wID':{'$ne': ''}}).distinct('wID')
	good_sessions = patient_ids
	return good_sessions	

def getEndpoints(morphline):    
    if morphline=='sedanMinivan':
        return ['sedan','minivan']
    elif morphline=='minivanSportscar':
        return ['minivan','sportscar']
    elif morphline=='sportscarSUV':
        return ['sportscar','SUV']
    elif morphline=='SUVMinivan':
        return ['SUV','minivan']
    elif morphline=='sportscarSedan':
        return ['sportscar','sedan']
    elif morphline=='sedanSUV':
        return ['sedan','SUV']
    elif morphline=='bedChair':
        return ['bed','chair']
    elif morphline=='bedTable':
        return ['bed','table']
    elif morphline=='benchBed':
        return ['bench','bed']
    elif morphline=='chairBench':
        return ['chair','bench']
    elif morphline=='chairTable':
        return ['chair','table']
    elif morphline=='tableBench':
        return ['table','bench']
    elif morphline=='limoToSUV':
        return ['limo','SUV']    
    elif morphline=='limoToSedan':
        return ['sedan','limo']  
    elif morphline=='limoToSmart':
        return ['limo','smartcar']  
    elif morphline=='smartToSedan':
        return ['smartcar','sedan']    
    elif morphline=='suvToSedan':
        return ['SUV','sedan']  
    elif morphline=='suvToSmart':
        return ['SUV','smartcar']  
    else:
        return ['A','B'] 

In [ ]:
def get_drawing_meta(w):

    import time
    start = time.time()

    # session parameters
    numPreRuns = 4
    numTrainRuns = 4
    numPostRuns = 2
    numTrialsTrainRun = 10
    numTrialsPreRun = 80
    numTrialsPreTotal = numTrialsPreRun*numPreRuns
    numTrialsTrainTotal = numTrialsTrainRun*numTrainRuns
    numTrials = numTrialsPreTotal*2 + numTrialsTrainTotal

    wID = []
    version = []
    category = []
    obj = []
    option1 = []
    option2 = []
    png = []
    viewpoint = []
    trialDuration = []
    phase = []
    trial = []
    counter = 0
    session_duration = []
    workers_complete = []
    TRnum = [] ## which TR of this block?
    runNum = [] ## function run index
    condition = []
    stim_onset = []

    try:
        these = coll.find({'wID': w}).sort('trialNum')   
        versionNum = these[0]['versionNum']
        design = [i for i in mdtd if i['version'] == int(versionNum)] # find which axes belong to which condition
        trained = design[0]['trained']
        near = design[0]['near']
        far1 = design[0]['far1']
        far2 = design[0]['far2']

        if these.count() > 9: 
            a = coll.find({'wID':w}).distinct('trialNum')
            upload_times = sort(map(math.trunc,map(float,a)))
            workers_complete.append(w)
            session_duration.append((max(upload_times)-min(upload_times))/1000/60)        
            counter = 0
            for ut in upload_times:
                ##aa = coll.find({'wID':w, 'trialNum': {'$in': [re.compile('.*' + str(ut) + '.*')]}}).sort('trialNum')
                aa = coll.find({'wID':w, 'trialNum': str(ut)}).sort('uploadDate')            
    #             ####=========######=======########===========##########==========##############
                if (aa[0]['task'] == 'drawing'): 
                    print 'Analyzing trial '  + str(ut) + ' from ' + aa[0]['wID']
                    assert aa.count()==1
                    wID.append(aa[0]['wID'])
                    version.append(int(aa[0]['versionNum']))
                    category.append(get_category_index(aa[0]['category']))
                    obj.append(get_object_name(2,get_object_index(aa[0]['morphline'],aa[0]['morphnum'])))
                    viewpoint.append(aa[0]['viewpoint'])
                    phase.append(get_phase_index(aa[0]['phase']))
                    trial.append(int(aa[0]['trialNum'])) 
                    TRnum.append(int(np.round((float(aa[0]['stimOnset'])-float(aa[0]['startBlock']))/1000/1.5)))  
                    stim_onset.append(float(aa[0]['stimOnset'])-float(aa[0]['startBlock']))
                    if float(aa[0]['trialNum'])==0:
                        runNum.append(1)
                    elif (float(aa[0]['trialNum'])>numTrialsPreTotal) & (float(aa[0]['trialNum']) <= (numTrialsPreTotal+numTrialsTrainTotal)):
                        runNum.append(ceil((float(aa[0]['trialNum'])-numTrialsPreTotal)/numTrialsTrainRun)+numPreRuns)
                    elif float(aa[0]['trialNum']) <= numTrialsPreTotal:
                        runNum.append(ceil((float(aa[0]['trialNum'])+1)/numTrialsPreRun))
                    elif float(aa[0]['trialNum']) >= numTrialsPreTotal+numTrialsTrainTotal:
                        runNum.append(ceil((float(aa[0]['trialNum'])-40+1)/numTrialsPreRun)+numTrainRuns)                                             
                    options = getEndpoints(aa[0]['morphline'])  
                    png.append(aa[0]['imgData'])
                    condition.append([trained,near,far1,far2].index(aa[0]['morphline'])) # 0=Trained, 1=Near, 2=Far1, 3=Far2

    except AssertionError,e:
        print str(e)
        pass

    X = pd.DataFrame([map(str,wID),map(int,version),map(str,category),map(str,obj),map(int,trial), \
                     TRnum,condition,stim_onset, map(int,runNum), map(int,viewpoint),png])
    X = X.transpose()
    X.columns = ['wID','versionNum','category','object', 'trial', \
                             'TRnum','condition','onset_time', 'run_num','viewpoint','png']

    
    ## unroll from trial-wise meta into TR-wise meta
    import copy
    morphrecog = copy.deepcopy(X)
    TR_unfurled = np.arange(1,max(TRnum))
    print "Now unrolling trial-wise meta into TR-wise meta ..."

    onset = np.zeros(len(morphrecog)).astype(int)
    morphrecog = morphrecog.assign(onset=pd.Series(onset).values)

    meta = []
    counter = 0
    numTrials =len(morphrecog)
    runStartDelayTR = 8 # number of TRs delay before first stim shown in each run
    num_TR_per_train_run = 286

    for counter in np.arange(numTrials):
        onset = morphrecog.iloc[counter]['TRnum']
        if onset != 8: ## NOT the first stim of a given run
            if counter < numTrials-1:        
                upcoming = morphrecog.loc[counter+1,'TRnum']
            else:
                upcoming = num_TR_per_train_run # number of TR's in training run
            soa = upcoming-onset
            if soa>0: # soa<0 after transition to new block
                _a = morphrecog.iloc[counter]
                __a = pd.Series.to_frame(_a)
                a = pd.concat([__a]*soa,axis=1,ignore_index=True).transpose()            
                a['TRnum']=np.arange(a.loc[0,'TRnum'],a.loc[0,'TRnum']+soa)
            else:
                reps = num_TR_per_train_run - onset
                _a = pd.Series.to_frame(morphrecog.iloc[counter])
                a = pd.concat([_a]*reps,axis=1,ignore_index=True).transpose()             
                a['TRnum']=np.arange(a.loc[0,'TRnum'],a.loc[0,'TRnum']+num_TR_per_train_run-onset)                
            a.loc[0,'onset'] = 1                    
            if len(meta)==0:
                meta = a
            else:
                meta = pd.concat([meta,a],ignore_index=True)            
        else:    ## the first stim of a given run   
            # tack on first 12 TR's at the top of meta, where no stim shown
            start = morphrecog.iloc[counter]['TRnum']
            _a = pd.Series.to_frame(morphrecog.iloc[counter])
            a = pd.concat([_a]*start,axis=1,ignore_index=True).transpose()                 
            a['TRnum'] = np.arange(a.loc[0,'TRnum'])
            a['condition'] = '-1'
            a['onset_time'] = 0
            if len(meta)==0:
                meta = a
            else:
                meta = pd.concat([meta,a],ignore_index=True)
            if counter < numTrials-1:        
                upcoming = morphrecog.loc[counter+1,'TRnum']
            else:
                upcoming = num_TR_per_train_run
            soa = upcoming-onset
            if soa>0: # soa<0 after transition to new block
                _a = pd.Series.to_frame(morphrecog.iloc[counter])
                a = pd.concat([_a]*soa,axis=1,ignore_index=True).transpose()                                      
                a['TRnum']=np.arange(a.loc[0,'TRnum'],a.loc[0,'TRnum']+soa)
            else:
                reps = num_TR_per_train_run-onset
                _a = pd.Series.to_frame(morphrecog.iloc[counter])
                a = pd.concat([_a]*reps,axis=1,ignore_index=True).transpose()                         
                a['TRnum']=np.arange(a.loc[0,'TRnum'],a.loc[0,'TRnum']+num_TR_per_train_run-onset)                
            a.loc[0,'onset'] = 1                    
            if len(meta)==0:
                meta = a
            else:
                meta = pd.concat([meta,a],ignore_index=True)
                
    end = time.time()
    elapsed = end - start
    print "Time taken: ", elapsed/60, "minutes."
        
    return X,meta

In [ ]:
X, meta = get_drawing_meta('1121161_neurosketch')

In [ ]:
def save_meta_to_csv(meta,w):
	meta.to_csv(os.path.join(DATADIR, w + '.csv'))

def get_meta_all_subs():
	workers = get_worker_list()
	for w in workers:
		print w
		X,meta = get_meta(w)
		save_meta_to_csv(meta,w)